# import

In [1]:
import sys
import os
import os.path as osp
import torch
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import pickle
import numpy as np

In [2]:
base_dir = '/workspace/CenterTrack/'
centertrack_path = osp.join(base_dir, 'src', 'lib')
sys.path.insert(0, centertrack_path)

In [3]:
from detector import Detector
from opts import opts

/home/ubuntu/.local/lib/python3.6/site-packages/sklearn/utils/linear_assignment_.py:22: FutureWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  FutureWarning)


# constant

In [4]:
TASK = 'tracking' # or 'tracking,multi_pose' for pose tracking and 'tracking,ddd' for monocular 3d tracking
MODEL_PATH = osp.join(base_dir, 'models', 'coco_tracking.pth')
# MODEL_PATH = osp.join(base_dir, 'models', 'mot17_fulltrain.pth')

In [5]:
opt = opts().init('{} --load_model {}'.format(TASK, MODEL_PATH).split(' '))

Running tracking
Using tracking threshold for out threshold! 0.3
Fix size testing.
training chunk_sizes: [32]
input h w: 512 512
heads {'hm': 80, 'reg': 2, 'wh': 2, 'tracking': 2}
weights {'hm': 1, 'reg': 1, 'wh': 0.1, 'tracking': 1}
head conv {'hm': [256], 'reg': [256], 'wh': [256], 'tracking': [256]}


In [11]:
opt.vis_thresh = 0.1

# path

In [6]:
data_dir = '../input/player/'

In [7]:
data_name = 'player'
data_dir = osp.join('../input', data_name)
save_dir = osp.join('../output', data_name, 'tracking')

# image_save_dir = osp.join('../output', data_name, 'image', 'person')
# label_save_dir = osp.join('../output', data_name, 'label')

# function

In [8]:
def listdirs(dir_path, name_condition=None):
    dir_names = os.listdir(dir_path)
    if name_condition is not None:
        dir_names = [dir_name for dir_name in dir_names if dir_name.startswith(name_condition)]
    dir_names.sort()
    return dir_names

In [9]:
def save_pickle(data, path):
    with open(path, 'wb') as f:
        pickle.dump(data, f)


def load_pickle(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return data

# main

In [10]:
person_class_id = 1

In [13]:
detector = Detector(opt)
input_meta = {'pre_dets': []}

games = listdirs(data_dir, 'game')
for game in games:
    print(game)
    game_dir = osp.join(data_dir, game)
    clips = listdirs(game_dir, 'Clip')
    for clip in tqdm(clips):
        clip_dir = osp.join(game_dir, clip)
        file_names = listdirs(clip_dir)
        label_names = [label for label in file_names if osp.splitext(label)[1]=='.json']
        image_names = [image for image in file_names if osp.splitext(image)[1]=='.jpg']
        
        results_df_list = []
        for frame_index, image_name in enumerate(image_names, 1):
            file_name = os.path.splitext(image_name)[0]
            image_path = osp.join(data_dir, game, clip, image_name)
            frame = cv2.imread(image_path)
            if frame_index==1:
                detector = Detector(opt)
                _ = detector.run(frame, input_meta)
                continue
            else:    
                results = detector.run(frame)['results']
                
                for result in results:
                    class_id = result['class']
                    if class_id != person_class_id:
                        continue
                    score = result['score']
                    tracking_id = result['tracking_id']
                    bbox = result['bbox']
                    
                    xmin, ymin, xmax, ymax = bbox.astype(int)
                    xmin = np.maximum(xmin, 0)
                    ymin = np.maximum(ymin, 0)
                    xmax = np.minimum(xmax, frame.shape[1])
                    ymax = np.minimum(ymax, frame.shape[0])
                    
                    frame_index = int(file_name)
                    label = [tracking_id, frame_index, xmin, ymin, xmax, ymax, score, image_name]
                    save_clip_dir = osp.join(save_dir, game, clip)
                                        
                    person_image = frame[ymin:ymax, xmin:xmax]
                    if person_image is None:
                        print('person_image is None')
                        continue
                    person_image_save_path = osp.join(save_clip_dir, '{:05}'.format(tracking_id), 'person_image', image_name)
                    os.makedirs(osp.dirname(person_image_save_path), exist_ok=True)
                    cv2.imwrite(person_image_save_path, person_image)
                    
                    label_save_path = osp.join(save_clip_dir, '{:05}'.format(tracking_id), 'label', '{}.pkl'.format(file_name))
                    os.makedirs(osp.dirname(label_save_path), exist_ok=True)
                    save_pickle(label, label_save_path)

Creating model...
Using node type: (<class 'model.networks.dla.DeformConv'>, <class 'model.networks.dla.DeformConv'>)
loaded /workspace/CenterTrack/models/coco_tracking.pth, epoch 70
Drop parameter base.pre_hm_layer.0.weight.
Drop parameter base.pre_hm_layer.1.weight.
Drop parameter base.pre_hm_layer.1.bias.
Drop parameter base.pre_hm_layer.1.running_mean.
Drop parameter base.pre_hm_layer.1.running_var.
Drop parameter base.pre_hm_layer.1.num_batches_tracked.




  0%|          | 0/13 [00:00<?, ?it/s]

game1
Creating model...
Using node type: (<class 'model.networks.dla.DeformConv'>, <class 'model.networks.dla.DeformConv'>)
loaded /workspace/CenterTrack/models/coco_tracking.pth, epoch 70
Drop parameter base.pre_hm_layer.0.weight.
Drop parameter base.pre_hm_layer.1.weight.
Drop parameter base.pre_hm_layer.1.bias.
Drop parameter base.pre_hm_layer.1.running_mean.
Drop parameter base.pre_hm_layer.1.running_var.
Drop parameter base.pre_hm_layer.1.num_batches_tracked.
Initialize tracking!




  8%|▊         | 1/13 [00:15<03:07, 15.61s/it]

Creating model...
Using node type: (<class 'model.networks.dla.DeformConv'>, <class 'model.networks.dla.DeformConv'>)
loaded /workspace/CenterTrack/models/coco_tracking.pth, epoch 70
Drop parameter base.pre_hm_layer.0.weight.
Drop parameter base.pre_hm_layer.1.weight.
Drop parameter base.pre_hm_layer.1.bias.
Drop parameter base.pre_hm_layer.1.running_mean.
Drop parameter base.pre_hm_layer.1.running_var.
Drop parameter base.pre_hm_layer.1.num_batches_tracked.
Initialize tracking!


...



 78%|███████▊  | 7/9 [01:56<00:27, 13.66s/it]

Creating model...
Using node type: (<class 'model.networks.dla.DeformConv'>, <class 'model.networks.dla.DeformConv'>)
loaded /workspace/CenterTrack/models/coco_tracking.pth, epoch 70
Drop parameter base.pre_hm_layer.0.weight.
Drop parameter base.pre_hm_layer.1.weight.
Drop parameter base.pre_hm_layer.1.bias.
Drop parameter base.pre_hm_layer.1.running_mean.
Drop parameter base.pre_hm_layer.1.running_var.
Drop parameter base.pre_hm_layer.1.num_batches_tracked.
Initialize tracking!




 89%|████████▉ | 8/9 [02:02<00:11, 11.26s/it]

Creating model...
Using node type: (<class 'model.networks.dla.DeformConv'>, <class 'model.networks.dla.DeformConv'>)
loaded /workspace/CenterTrack/models/coco_tracking.pth, epoch 70
Drop parameter base.pre_hm_layer.0.weight.
Drop parameter base.pre_hm_layer.1.weight.
Drop parameter base.pre_hm_layer.1.bias.
Drop parameter base.pre_hm_layer.1.running_mean.
Drop parameter base.pre_hm_layer.1.running_var.
Drop parameter base.pre_hm_layer.1.num_batches_tracked.
Initialize tracking!




100%|██████████| 9/9 [02:22<00:00, 15.87s/it]
